<a href="https://colab.research.google.com/github/enesemretas/mcpath-colab/blob/main/notebooks/mcpath_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# @title MCPath (User Interface Only)
# This cell only loads the UI from the GitHub module (no debug output).
!pip -q install pyyaml ipywidgets requests

import sys, subprocess, pathlib

REPO_URL = "https://github.com/enesemretas/mcpath-colab.git"
REPO_DIR = pathlib.Path("/content/mcpath-colab")

# --- Clean clone or update (quiet) ---
if REPO_DIR.exists():
    subprocess.run(["git", "-C", str(REPO_DIR), "pull", "-q"], check=True)
else:
    subprocess.run(["git", "clone", "--depth", "1", REPO_URL, str(REPO_DIR)], check=True)

sys.path.append(str(REPO_DIR))

from mcpath.ui import launch
launch(
    defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
    show_title="MCPath-style Parameters"
)
